In [1]:
import os
os.chdir("/home/yizhou7/mariposa")

from debugger.demo_utils import *
from debugger3 import Debugger3
from debugger.tree_node import NodeRef

from pandas import DataFrame 
import networkx as nx


In [2]:
q = "data/projs/bench_unstable/base.z3/d_fvbkv--Impl-LeafImpl.i.dfy.Impl__LeafImpl.__default.repivotLeaf.smt2"
r = Reviewer2(q)
report = r.get_report()
editor = r.dbg.editor

report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
report.freq["stabilized"] = report.freq["qname"].isin(report.stabilized["qname"])
# s = report.freq.sort_values(by="rank")

[INFO] [init] query path: data/projs/bench_unstable/base.z3/d_fvbkv--Impl-LeafImpl.i.dfy.Impl__LeafImpl.__default.repivotLeaf.smt2 
[INFO] [init] dbg root: dbg/2298e56311 


[DEBUG] using cache at 2298e56311.report 
[DEBUG] [edit] proof path: dbg/2298e56311/proofs/reseed.14633177203038070573.proof 
[DEBUG] [edit] trace path: dbg/2298e56311/traces/reseed.6910845141689289050 
[DEBUG] using cache at b4eb0e53f8c23a334df1af9711c99f98.pickle 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/2298e56311/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/2298e56311/orig.smt2 


In [3]:
print(tabulate(report.tested, headers='keys', tablefmt='psql'))

+-----+--------------------+--------------+----------+--------+-------------------------------------------------------+
|     | qname              | action       | result   |   time | edit_path                                             |
|-----+--------------------+--------------+----------+--------+-------------------------------------------------------|
|   0 | mariposa_qid_13881 | erase        | unsat    | 31.702 | data/projs/singleton_2298e56311/base.z3/0038a9cc.smt2 |
|   1 | mariposa_qid_338   | inst_keep    | unsat    | 33.414 | data/projs/singleton_2298e56311/base.z3/00e0383f.smt2 |
|   2 | mariposa_qid_3984  | erase        | unsat    | 30.205 | data/projs/singleton_2298e56311/base.z3/012d9372.smt2 |
|   3 | mariposa_qid_1556  | inst_replace | unsat    | 24.971 | data/projs/singleton_2298e56311/base.z3/013ad49f.smt2 |
|   4 | mariposa_qid_6266  | inst_replace | unsat    | 31.155 | data/projs/singleton_2298e56311/base.z3/01affac4.smt2 |
|   5 | mariposa_qid_4024  | erase      

In [4]:
print(tabulate(report.stabilized, headers='keys', tablefmt='psql'))

+----+-------------------+--------------+-------------------------------------------------------+
|    | qname             | action       | edit_path                                             |
|----+-------------------+--------------+-------------------------------------------------------|
|  0 | mariposa_qid_1491 | inst_replace | data/projs/singleton_2298e56311/base.z3/7c2d8b54.smt2 |
|  1 | mariposa_qid_1491 | inst_keep    | data/projs/singleton_2298e56311/base.z3/d51a540e.smt2 |
+----+-------------------+--------------+-------------------------------------------------------+


In [5]:
# editor.debug_qanme("mariposa_qid_1491")
proof = editor.proof
proof_graph = proof.proof_graph

print(len(proof_graph.nodes))
print(len(proof.nodes))

36315
898389


In [11]:
in_degrees = proof_graph.in_degree()
in_degrees = sorted(in_degrees, key=lambda x: x[1], reverse=True)

parent_in_degrees = {}
qnames = {}

for nr, degree in in_degrees:
    node = proof.lookup_node(nr)
    if node.name != "quant-inst":
        continue
    assert degree == 1
    # proof.pprint_node(node, 3)
    qid = proof.get_qname_of_inst_ref(nr)
    parent = list(proof_graph.predecessors(nr))[0]
    parent_in_degrees[parent] = proof_graph.in_degree(parent)
    qnames[parent] = qid

parent_in_degrees = sorted(parent_in_degrees.items(), key=lambda x: x[1], reverse=True)

for nr, degree in parent_in_degrees:
    node = proof.lookup_node(nr)
    print(degree, qnames[nr], len(nx.shortest_path(proof_graph, source=proof.root_ref, target=nr)))
    proof.pprint_node(node, 0)

112 mariposa_qid_17 24
(unit-resolution @[h!b5af3257108f81ec] @[h!c74d1435be2135c1] @[h!a480cadcd5541b53])
70 mariposa_qid_50 18
(unit-resolution @[h!c3b09faf7c3b7045] @[h!ba732cdf75e0fe8c] @[h!1a46da3d28996e0d])
49 mariposa_qid_20 23
(unit-resolution @[h!a68ad26fe744eba5] @[h!ecb74b9488cab304] @[h!d58ced9df4be9ab6])
49 mariposa_qid_14240 4
(unit-resolution @[h!4e4abac480d451c6] @[h!73cff31402142d67] @[h!b7b124c12cb74ca4])
39 mariposa_qid_71 29
(unit-resolution @[h!4854bc9f647ad81d] @[h!42bfc201dde79ca8] @[h!2bbd00e39ce6b8cb])
38 mariposa_qid_72 25
(unit-resolution @[h!d040855ef0ce4d11] @[h!cd0eff11d422a230] @[h!a37f0f589c17d1df])
37 mariposa_qid_14064 27
(unit-resolution @[h!8d8c2499879a437a] @[h!fba013c8c5460b6f] @[h!8043f805b41ae71c])
35 mariposa_qid_14241 27
(unit-resolution @[h!57a720e5bb9364c6] @[h!46f66cd1875e411f] @[h!9d9d40d0b213dc2c])
33 mariposa_qid_13946 11
(unit-resolution @[h!484dcc5f3c41f64d] @[h!25971202d3215a38] @[h!e43d8f903e753c4d])
32 mariposa_qid_14241 7
(unit-reso